In [1]:
import sys
import os
import sqlite3 as sq
import pandas as pd

In [20]:
InputFileName1='Retrieve-Country-Currency.csv'
InputFileName2='Retrieve_Euro_EchangeRates.csv'

InputFileDir1='C:/VKHCG/01-Vermeulen/01-Retrieve/01-EDS/02-Python'
InputFileDir2='C:/VKHCG/04-Clark/01-Retrieve/01-EDS/01-R'

#OutputFileName=
OutputFileDir = 'C:/VKHCG/04-Clark/02-Assess/01-EDS/02-Python'

DataBaseName='clark.db'
DataBaseDir = 'C:/VKHCG/02-Krennwallner/02-Assess/SQLite'

In [11]:
sDatabaseName=DataBaseDir + '/' + DataBaseName
conn = sq.connect(sDatabaseName)

In [21]:
sFileName1=InputFileDir1 + '/' + InputFileName1
print('################################')
print('Loading :',sFileName1)

################################
Loading : C:/VKHCG/01-Vermeulen/01-Retrieve/01-EDS/02-Python/Retrieve-Country-Currency.csv


In [22]:
CountryRawData=pd.read_csv(sFileName1,header=0,low_memory=False, encoding="latin-1")
CountryRawData.drop_duplicates(subset=None, keep='first', inplace=True)
CountryData=CountryRawData
print('Loaded Company :',CountryData.columns.values)

Loaded Company : ['Country' 'Currency' 'CurrencyCode']


In [23]:
sTable='Assess_Country'
print('Storing :',sDatabaseName,' Table:',sTable)
CountryData.to_sql(sTable, conn, if_exists="replace")

Storing : C:/VKHCG/04-Clark/01-Retrieve/01-EDS/02-Python/clark.db  Table: Assess_Country


In [24]:
print(CountryData.head())
print('Rows : ',CountryData.shape[0])

                      Country        Currency CurrencyCode
0                 Afghanistan  Afghan afghani          AFN
1  Akrotiri and Dhekelia (UK)   European euro          EUR
2     Aland Islands (Finland)   European euro          EUR
3                     Albania    Albanian lek          ALL
4                     Algeria  Algerian dinar          DZD
Rows :  253


In [25]:
### Import Forex Data
sFileName2=InputFileDir2 + '/' + InputFileName2
print('################################')
print('Loading :',sFileName2)

################################
Loading : C:/VKHCG/04-Clark/01-Retrieve/01-EDS/01-R/Retrieve_Euro_EchangeRates.csv


In [28]:
ForexRawData=pd.read_csv(sFileName2,header=0,low_memory=False, encoding="latin-1")
ForexRawData.drop_duplicates(subset=None, keep='first', inplace=True)
ForexData=ForexRawData.head(5)

In [29]:
print('Rows : ',ForexData.shape[0])
print('Loaded Company :',ForexData.columns.values)

Rows :  5
Loaded Company : ['Date' 'CodeIn' 'CodeOut' 'Rate']


In [30]:
sTable='Assess_Forex'
print('Storing :',sDatabaseName,' Table:',sTable)
ForexData.to_sql(sTable, conn, if_exists="replace")

Storing : C:/VKHCG/04-Clark/01-Retrieve/01-EDS/02-Python/clark.db  Table: Assess_Forex


In [31]:
print(ForexData.head())
print('Rows : ',ForexData.shape[0])

         Date CodeIn CodeOut      Rate
0  2017-05-05    AUD     AUD  1.000000
1  2017-05-05    AUD     BGN  1.318635
2  2017-05-05    AUD     BRL  2.354167
3  2017-05-05    AUD     CAD  1.017665
4  2017-05-05    AUD     CHF  0.730785
Rows :  5


In [32]:
sTable='Assess_Forex'
print('Loading :',sDatabaseName,' Table:',sTable)
sSQL="select distinct"
sSQL=sSQL+ " A.CodeIn"
sSQL=sSQL+ " from"
sSQL=sSQL+ " Assess_Forex as A;"
CodeData=pd.read_sql_query(sSQL, conn)

Loading : C:/VKHCG/04-Clark/01-Retrieve/01-EDS/02-Python/clark.db  Table: Assess_Forex


In [33]:
sSQL

'select distinct A.CodeIn from Assess_Forex as A;'

In [36]:
for c in range(CodeData.shape[0]):
    print('################')  
    sTable='Assess_Forex & 2x Country > ' + CodeData['CodeIn'][c]
    print('Loading :',sDatabaseName,' Table:',sTable)
    sSQL="select distinct"
    sSQL=sSQL+ " A.Date,"
    sSQL=sSQL+ " A.CodeIn,"
    sSQL=sSQL+ " B.Country as CountryIn,"
    sSQL=sSQL+ " B.Currency as CurrencyNameIn,"
    sSQL=sSQL+ " A.CodeOut,"
    sSQL=sSQL+ " C.Country as CountryOut,"
    sSQL=sSQL+ " C.Currency as CurrencyNameOut,"
    sSQL=sSQL+ " A.Rate"
    sSQL=sSQL+ " from"
    sSQL=sSQL+ " Assess_Forex as A"
    sSQL=sSQL+ " JOIN"
    sSQL=sSQL+ " Assess_Country as B"
    sSQL=sSQL+ " ON A.CodeIn = B.CurrencyCode"
    sSQL=sSQL+ " JOIN"
    sSQL=sSQL+ " Assess_Country as C"
    sSQL=sSQL+ " ON A.CodeOut = C.CurrencyCode"
    sSQL=sSQL+ " WHERE"
    sSQL=sSQL+ " A.CodeIn ='" + CodeData['CodeIn'][c] + "';"
    
    

################
Loading : C:/VKHCG/04-Clark/01-Retrieve/01-EDS/02-Python/clark.db  Table: Assess_Forex & 2x Country > AUD
################
          Date CodeIn                   CountryIn     CurrencyNameIn CodeOut  \
0   2017-05-05    AUD                   Australia  Australian dollar     AUD   
1   2017-05-05    AUD                   Australia  Australian dollar     AUD   
2   2017-05-05    AUD                   Australia  Australian dollar     AUD   
3   2017-05-05    AUD                   Australia  Australian dollar     AUD   
4   2017-05-05    AUD                   Australia  Australian dollar     AUD   
..         ...    ...                         ...                ...     ...   
79  2017-05-05    AUD                       Nauru  Australian dollar     CHF   
80  2017-05-05    AUD  Norfolk Island (Australia)  Australian dollar     CHF   
81  2017-05-05    AUD  Norfolk Island (Australia)  Australian dollar     CHF   
82  2017-05-05    AUD                      Tuvalu  Australia

In [35]:
    ForexData=pd.read_sql_query(sSQL, conn).head(1000)
    print('################')  
    print(ForexData)
    print('################')  
    sTable='Assess_Forex_' + CodeData['CodeIn'][c] 
    print('Storing :',sDatabaseName,' Table:',sTable)
    ForexData.to_sql(sTable, conn, if_exists="replace")
    print('################')  
    print('################################')
    print('Rows : ',ForexData.shape[0])

################
          Date CodeIn                   CountryIn     CurrencyNameIn CodeOut  \
0   2017-05-05    AUD                   Australia  Australian dollar     AUD   
1   2017-05-05    AUD                   Australia  Australian dollar     AUD   
2   2017-05-05    AUD                   Australia  Australian dollar     AUD   
3   2017-05-05    AUD                   Australia  Australian dollar     AUD   
4   2017-05-05    AUD                   Australia  Australian dollar     AUD   
..         ...    ...                         ...                ...     ...   
79  2017-05-05    AUD                       Nauru  Australian dollar     CHF   
80  2017-05-05    AUD  Norfolk Island (Australia)  Australian dollar     CHF   
81  2017-05-05    AUD  Norfolk Island (Australia)  Australian dollar     CHF   
82  2017-05-05    AUD                      Tuvalu  Australian dollar     CHF   
83  2017-05-05    AUD                      Tuvalu  Australian dollar     CHF   

                      